# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb31c2c91c0>
├── 'a' --> tensor([[-1.3325, -0.8603, -0.3516],
│                   [-0.4515, -0.4454, -0.7691]])
└── 'x' --> <FastTreeValue 0x7fb31c2fbdf0>
    └── 'c' --> tensor([[-0.6316, -1.1939,  1.1927,  0.6673],
                        [-0.1404, -1.0289, -0.0509, -0.9646],
                        [ 1.5470, -1.5546,  0.1414,  1.5432]])

In [4]:
t.a

tensor([[-1.3325, -0.8603, -0.3516],
        [-0.4515, -0.4454, -0.7691]])

In [5]:
%timeit t.a

62.1 ns ± 0.0578 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb31c2c91c0>
├── 'a' --> tensor([[ 0.7224,  0.6305,  1.0489],
│                   [ 0.7134, -0.3695,  0.9253]])
└── 'x' --> <FastTreeValue 0x7fb31c2fbdf0>
    └── 'c' --> tensor([[-0.6316, -1.1939,  1.1927,  0.6673],
                        [-0.1404, -1.0289, -0.0509, -0.9646],
                        [ 1.5470, -1.5546,  0.1414,  1.5432]])

In [7]:
%timeit t.a = new_value

72.5 ns ± 0.0274 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3325, -0.8603, -0.3516],
               [-0.4515, -0.4454, -0.7691]]),
    x: Batch(
           c: tensor([[-0.6316, -1.1939,  1.1927,  0.6673],
                      [-0.1404, -1.0289, -0.0509, -0.9646],
                      [ 1.5470, -1.5546,  0.1414,  1.5432]]),
       ),
)

In [10]:
b.a

tensor([[-1.3325, -0.8603, -0.3516],
        [-0.4515, -0.4454, -0.7691]])

In [11]:
%timeit b.a

54.7 ns ± 0.0225 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.4331,  1.1038,  0.1392],
               [-0.4199, -0.5017, -0.7423]]),
    x: Batch(
           c: tensor([[-0.6316, -1.1939,  1.1927,  0.6673],
                      [-0.1404, -1.0289, -0.0509, -0.9646],
                      [ 1.5470, -1.5546,  0.1414,  1.5432]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.0572 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 0.159 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 23.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 476 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb26ef61520>
├── 'a' --> tensor([[[-1.3325, -0.8603, -0.3516],
│                    [-0.4515, -0.4454, -0.7691]],
│           
│                   [[-1.3325, -0.8603, -0.3516],
│                    [-0.4515, -0.4454, -0.7691]],
│           
│                   [[-1.3325, -0.8603, -0.3516],
│                    [-0.4515, -0.4454, -0.7691]],
│           
│                   [[-1.3325, -0.8603, -0.3516],
│                    [-0.4515, -0.4454, -0.7691]],
│           
│                   [[-1.3325, -0.8603, -0.3516],
│                    [-0.4515, -0.4454, -0.7691]],
│           
│                   [[-1.3325, -0.8603, -0.3516],
│                    [-0.4515, -0.4454, -0.7691]],
│           
│                   [[-1.3325, -0.8603, -0.3516],
│                    [-0.4515, -0.4454, -0.7691]],
│           
│                   [[-1.3325, -0.8603, -0.3516],
│                    [-0.4515, -0.4454, -0.7691]]])
└── 'x' --> <FastTreeValue 0x7fb26ef61e50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 66 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb31c295220>
├── 'a' --> tensor([[-1.3325, -0.8603, -0.3516],
│                   [-0.4515, -0.4454, -0.7691],
│                   [-1.3325, -0.8603, -0.3516],
│                   [-0.4515, -0.4454, -0.7691],
│                   [-1.3325, -0.8603, -0.3516],
│                   [-0.4515, -0.4454, -0.7691],
│                   [-1.3325, -0.8603, -0.3516],
│                   [-0.4515, -0.4454, -0.7691],
│                   [-1.3325, -0.8603, -0.3516],
│                   [-0.4515, -0.4454, -0.7691],
│                   [-1.3325, -0.8603, -0.3516],
│                   [-0.4515, -0.4454, -0.7691],
│                   [-1.3325, -0.8603, -0.3516],
│                   [-0.4515, -0.4454, -0.7691],
│                   [-1.3325, -0.8603, -0.3516],
│                   [-0.4515, -0.4454, -0.7691]])
└── 'x' --> <FastTreeValue 0x7fb2f0700e50>
    └── 'c' --> tensor([[-0.6316, -1.1939,  1.1927,  0.6673],
                        [-0.1404, -1.0289, -0.0509, -0.9646],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.6 µs ± 95.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.3325, -0.8603, -0.3516],
                [-0.4515, -0.4454, -0.7691]],
       
               [[-1.3325, -0.8603, -0.3516],
                [-0.4515, -0.4454, -0.7691]],
       
               [[-1.3325, -0.8603, -0.3516],
                [-0.4515, -0.4454, -0.7691]],
       
               [[-1.3325, -0.8603, -0.3516],
                [-0.4515, -0.4454, -0.7691]],
       
               [[-1.3325, -0.8603, -0.3516],
                [-0.4515, -0.4454, -0.7691]],
       
               [[-1.3325, -0.8603, -0.3516],
                [-0.4515, -0.4454, -0.7691]],
       
               [[-1.3325, -0.8603, -0.3516],
                [-0.4515, -0.4454, -0.7691]],
       
               [[-1.3325, -0.8603, -0.3516],
                [-0.4515, -0.4454, -0.7691]]]),
    x: Batch(
           c: tensor([[[-0.6316, -1.1939,  1.1927,  0.6673],
                       [-0.1404, -1.0289, -0.0509, -0.9646],
                       [ 1.5470, -1.5546,  0.1414,  1.5432]],
         

In [26]:
%timeit Batch.stack(batches)

78 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.3325, -0.8603, -0.3516],
               [-0.4515, -0.4454, -0.7691],
               [-1.3325, -0.8603, -0.3516],
               [-0.4515, -0.4454, -0.7691],
               [-1.3325, -0.8603, -0.3516],
               [-0.4515, -0.4454, -0.7691],
               [-1.3325, -0.8603, -0.3516],
               [-0.4515, -0.4454, -0.7691],
               [-1.3325, -0.8603, -0.3516],
               [-0.4515, -0.4454, -0.7691],
               [-1.3325, -0.8603, -0.3516],
               [-0.4515, -0.4454, -0.7691],
               [-1.3325, -0.8603, -0.3516],
               [-0.4515, -0.4454, -0.7691],
               [-1.3325, -0.8603, -0.3516],
               [-0.4515, -0.4454, -0.7691]]),
    x: Batch(
           c: tensor([[-0.6316, -1.1939,  1.1927,  0.6673],
                      [-0.1404, -1.0289, -0.0509, -0.9646],
                      [ 1.5470, -1.5546,  0.1414,  1.5432],
                      [-0.6316, -1.1939,  1.1927,  0.6673],
                      [-0.1404, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 335 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 3.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
